# Phase Gradient

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `AddIntoPhaseGrad`
Quantum-quantum addition into a phase gradient register using $b_{phase} - 2$ Toffolis

$$
    U|x\rangle|\text{phase\_grad}\rangle = |x\rangle|\text{phase\_grad} + x\rangle
$$

#### Parameters
 - `x_bitsize`: Size of input register.
 - `phase_bitsize`: Size of phase gradient register to which the input value should be added.
 - `right_shift`: An integer specifying the amount by which the input register x should be right shifted before adding to the phase gradient register.
 - `sign`: Whether the input register x should be  added or subtracted from the phase gradient register.
 - `controlled`: Whether to control this bloq with a ctrl register. When controlled=None, this bloq is not controlled. When controlled=0, this bloq is active when the ctrl register is 0. When controlled=1, this bloq is active when the ctrl register is 1. 

#### Registers
 - `- ctrl`: Control THRU register
 - `- x`: Input THRU register storing input value x to be added to the phase gradient register.
 - `- phase_grad`: Phase gradient THRU register. 

#### References
 - [Compilation of Fault-Tolerant Quantum Heuristics for Combinatorial Optimization](https://arxiv.org/abs/2007.07391). Appendix A: Addition for controlled rotations


In [ ]:
from qualtran.bloqs.rotations import AddIntoPhaseGrad

### Example Instances

In [ ]:
add_into_phase_grad = AddScaledValIntoPhaseReg.from_bitsize(4, 7, 0.123, 6)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([add_into_phase_grad],
           ['`add_into_phase_grad`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
add_into_phase_grad_g, add_into_phase_grad_sigma = add_into_phase_grad.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(add_into_phase_grad_g)
show_counts_sigma(add_into_phase_grad_sigma)